In [206]:
import pinocchio as pin
import numpy as np
import hppfcl as fcl
from pinocchio.visualize import MeshcatVisualizer as Visualizer
import time
import meshcat
from ipywidgets import interact
import ipywidgets as widgets

In [323]:
parent_id = 0
arm_mass = 0.7
upper_arm_radius = 2e-2
upper_arm_length = 0.25
lower_arm_radius = 1e-2
lower_arm_length = 0.25
axis_length = 0.1 # length of the co-ordinate axis rectangle


rmodel = pin.Model()
rmodel.name = "Human Arm"
gmodel = pin.GeometryModel()

# Joint
joint_name = "Shoulder_z"
Base_placement = pin.SE3.Identity()
Base_placement.rotation = pin.utils.rpyToMatrix(0,0,np.pi/2.0)
Sz = rmodel.addJoint(parent_id, pin.JointModelRZ(), Base_placement, joint_name)

joint_name = "Shoulder_x"
Base_placement = pin.SE3.Identity()
Sx = rmodel.addJoint(Sz, pin.JointModelRX(), Base_placement, joint_name)

frame_name = "imu_arm"
imu_arm_placement = pin.SE3.Identity()
imu_arm_placement.rotation = pin.utils.rpyToMatrix(0,np.pi/2.0,0) @ pin.utils.rpyToMatrix(np.pi, 0, 0) 
imu_arm_placement.translation[2] = -upper_arm_length/2.0
imu_arm_frame = pin.Frame(frame_name, Sx, parent_id, imu_arm_placement, pin.OP_FRAME)
rmodel.addFrame(imu_arm_frame)

joint_name = "upper_arm_rotation"
uar = pin.SE3.Identity()
uar.rotation = pin.utils.rpyToMatrix(0,0,np.pi/2.0)
uar.translation[2] = -lower_arm_length/2
uar_id = rmodel.addJoint(Sx, pin.JointModelRZ(), uar, joint_name)

joint_name = "Elbow"
elbow_placement = pin.SE3.Identity()
elbow_placement.translation[2] = -upper_arm_length/2.0
elbow_id = rmodel.addJoint(uar_id, pin.JointModelRY(), elbow_placement, joint_name)

joint_name = "lower_arm_rotation"
lar = pin.SE3.Identity()
lar.translation[2] = -lower_arm_length/2
lar_id = rmodel.addJoint(elbow_id, pin.JointModelRZ(), lar, joint_name)

frame_name = "Hand"
hand_placement = pin.SE3.Identity()
hand_placement.rotation = pin.utils.rpyToMatrix(0,np.pi/2.0,0.0) @ pin.utils.rpyToMatrix(-np.pi,0.0, 0)
hand_placement.translation[2] = -lower_arm_length/2.0
hand_frame = pin.Frame(frame_name, lar_id, lar_id, hand_placement, pin.OP_FRAME)
hand_id = rmodel.addFrame(hand_frame)

# Upper Arm Half
upper_arm1_inertia = pin.Inertia.FromCylinder(arm_mass/2.0, upper_arm_radius, upper_arm_length/2.0)
upper_arm1_placement = Base_placement.copy()
upper_arm1_placement.translation[2] = -upper_arm_length/4.0
rmodel.appendBodyToJoint(Sx, upper_arm1_inertia, upper_arm1_placement)

geom_name = "Upper Arm1"
shape = fcl.Cylinder(upper_arm_radius, upper_arm_length/2.0)
shape_placement = upper_arm1_placement.copy()

geom_obj = pin.GeometryObject(geom_name, Sx, shape, shape_placement)
geom_obj.meshColor = np.array([1.,1.,1.,1.])
gmodel.addGeometryObject(geom_obj)

# Upper Arm2
upper_arm2_inertia = pin.Inertia.FromCylinder(arm_mass/2, lower_arm_radius, upper_arm_length/2)
upper_arm2_placement = uar.copy()
upper_arm2_placement.translation[2] = -lower_arm_length/4.0
rmodel.appendBodyToJoint(uar_id, upper_arm2_inertia, upper_arm2_placement)

geom_name = "Upper Arm2"
shape = fcl.Cylinder(lower_arm_radius, lower_arm_length/2.0)
shape_placement = upper_arm2_placement.copy()

lower_arm_obj = pin.GeometryObject(geom_name, uar_id, shape, shape_placement)
lower_arm_obj.meshColor = np.array([1.,1.,0.,1.])
gmodel.addGeometryObject(lower_arm_obj)

# Lower Arm 1
lower_arm_inertia = pin.Inertia.FromCylinder(arm_mass/2.0, lower_arm_radius, lower_arm_length/2.0)
lower_arm_placement = elbow_placement.copy()
lower_arm_placement.translation[2] = -lower_arm_length/4.0
rmodel.appendBodyToJoint(elbow_id, lower_arm_inertia, lower_arm_placement)

geom_name = "Lower Arm1"
shape = fcl.Cylinder(1.5*lower_arm_radius, lower_arm_length/2.0)
shape_placement = lower_arm_placement.copy()

lower_arm_obj1 = pin.GeometryObject(geom_name, elbow_id, shape, shape_placement)
lower_arm_obj1.meshColor = np.array([1.,0.,0.,1.])
gmodel.addGeometryObject(lower_arm_obj1)

# Lower Arm 2
lower_arm_inertia2 = pin.Inertia.FromCylinder(arm_mass/2.0, lower_arm_radius, lower_arm_length/2.0)
lower_arm_placement2 = elbow_placement.copy()
lower_arm_placement2.translation[2] = -lower_arm_length/4.0
rmodel.appendBodyToJoint(lar_id, lower_arm_inertia, lower_arm_placement)

geom_name = "Lower Arm2"
shape = fcl.Cylinder(lower_arm_radius, lower_arm_length/2.0)
shape_placement = lower_arm_placement2.copy()

lower_arm_obj2 = pin.GeometryObject(geom_name, lar_id, shape, shape_placement)
lower_arm_obj2.meshColor = np.array([1.,1.,0.,1.])
gmodel.addGeometryObject(lower_arm_obj2)


# Hand 
shape_base = fcl.Box(0.05,0.05,0.05)
placement = pin.SE3.Identity()
placement.translation[2] = -lower_arm_length/2.0 - 0.05/2.0
hand = pin.GeometryObject("hand", hand_id, lar_id, shape_base, placement)
hand.meshColor = np.array([0.,1.0,1.0,1.])
gmodel.addGeometryObject(hand)


cmodel = gmodel
rdata = rmodel.createData()
gdata = gmodel.createData()


In [324]:
viz = Visualizer(rmodel, gmodel, cmodel)
viewer = meshcat.Visualizer(zmq_url = "tcp://127.0.0.1:6000")

viz.initViewer(viewer)
viz.loadViewerModel()
viz.initializeFrames()
viz.display_frames = True

# viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [325]:
# q = np.array([0.5, 0.5, 0.5])
# viz.display(q)

In [326]:
def move_joints(j1, j2, j3, j4, j5):
    q = pin.neutral(rmodel)
    q[0] = j1
    q[1] = j2
    q[2] = j3
    q[3] = j4
    q[4] = j5
    viz.display(q)

In [327]:
interact(move_joints,
         j1=(-2.,2), j2=(-5.,5), j3=(-2.,2), j4=(-2.,2), j5=(-2.,2))

interactive(children=(FloatSlider(value=0.0, description='j1', max=2.0, min=-2.0), FloatSlider(value=0.0, desc…

<function __main__.move_joints(j1, j2, j3, j4, j5)>

In [307]:
# q = np.array([0.0, -1.0,-1.5, 0.0, -np.pi/2.0])
q = np.array([-1.5,0.0,0, 0.0, -np.pi/2.0])

pin.framesForwardKinematics(rmodel, rdata, q)
pin.updateFramePlacements(rmodel, rdata)
print(rdata.oMf[rmodel.getFrameId("imu_arm")].rotation)
viz.display_frames = True
viz.display(q)
time.sleep(0.1)

[[-3.14136280e-17  7.07372017e-02  9.97494987e-01]
 [ 4.42976760e-16 -9.97494987e-01  7.07372017e-02]
 [ 1.00000000e+00  4.44089210e-16  4.93038066e-32]]


In [104]:
# q = np.array([-np.pi/4.0,-np.pi/4.0,0.0,0.0, -np.pi/2.0])
q = np.zeros(rmodel.nq)
v = np.zeros(rmodel.nq)
tau = np.zeros(rmodel.nq)
dt = 0.001
for i in range(1000):
    time.sleep(0.001)
    tau[0] = 0.1
    tau[1] = 0.8
    a = pin.aba(rmodel, rdata, q, v, tau)
    v += a*dt
    q += v*dt
    viz.display(q)